In [75]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from broyden import *
import time

from sympy.utilities.lambdify import lambdify, implemented_function
from scipy import optimize

Pi = sym.symbols('pi')
n0 = 0.153
hc = 197.326980 # MeV fm

def sqrt(n):
    return np.sqrt(n)

def log(n):
    return np.log(n)

In [37]:
class eos:
    def __init__(self, g_sigma, g_omega, g_rho = 0.0, b = 0.0, c = 0.0):
        
        self.g_sigma = g_sigma
        self.g_omega = g_omega
        self.g_rho = g_rho
        self.b = b
        self.c = c

class baryon:
    def __init__(self, mass, spin, isospin, charge, kind, var_type, mass_eff = 0.0, num_density = 0.0,\
                 frac = 0.0, kf = 0.0, ef = 0.0, chem_pot = 0.0):
    
        # variables to be established at baryon declaration
        self.mass = mass
        self.spin = spin 
        self.isospin = isospin
        self.charge = charge
        self.kind = kind
        self.var_type = var_type
    
        # variables to be stored later
        self.mass_eff = mass_eff
        self.num_density = num_density
        self.frac = frac
        self.kf = kf
        self.ef = ef
        self.chem_pot = chem_pot
        
        # other things
        self.g_sigma = 0.0
        self.g_omega = 0.0 
        self.g_rho = 0.0 
        self.g_phi = 0.0 

class lepton:
    def __init__(self, mass, charge, num_density = 0.0, frac = 0.0, var_type = 0.0, kf = 0.0, chem_pot = 0.0):
        self.mass = mass
        self.charge = charge
        self.num_density = num_density
        self.frac = frac
        self.var_type = var_type
        self.kf = kf
        self.chem_pot = chem_pot

class meson:
    def __init__(self, mass, field = 0.0):
        self.mass = mass # in MeV
        self.field = field

In [38]:
# glen denning constants
rmf = eos(g_sigma = 8.79509376389, g_omega = 9.1815177, g_rho = 9.7793745, b = 0.00414, c = 0.00716)

In [39]:
# electron
electron = lepton(0.510, -1)

# proton 
proton = baryon(939.0, 1/2, 1/2, 1, 'Nucleon', 'Dependent')

# neutron 
neutron = baryon(939.0, 1/2, -1/2, 0, 'Nucleon', 'Dependent')

In [40]:
proton.g_sigma = rmf.g_sigma
proton.g_omega = rmf.g_omega
proton.g_rho = rmf.g_rho

neutron.g_sigma = rmf.g_sigma
neutron.g_omega = rmf.g_omega
neutron.g_rho = rmf.g_rho

In [41]:
# declaring the numeric meson objects
sigma = meson(550.0)
omega = meson(783.0)
rho = meson(770.0)

In [42]:
baryon_list = [proton, neutron]
meson_list = [sigma, omega, rho]
lepton_list = [electron]

In [86]:
nb_orig = 0.27
nb = nb_orig*0.153*hc**3

### Defining Things

In [44]:
def dU(sig):
    # derivative dU/dsigma as a func of sigma
    return rmf.b*rmf.g_sigma**3*neutron.mass*sig**2 + rmf.c*rmf.g_sigma**4*sig**3

In [45]:
def proton_mass_eff(sig):
    return proton.mass - rmf.g_sigma*sig

def neutron_mass_eff(sig):
    return neutron.mass - rmf.g_sigma*sig

def mass_eff(baryon, sig):
    return baryon.mass - rmf.g_sigma*sig

In [46]:
def kf(frac):
    return (3*np.pi**2*nb*frac)**(1/3)

42.20609737857957

In [53]:
def proton_e_eff(frac, sig):
    return np.sqrt(kf(frac)**2 + proton_mass_eff(sig)**2)

def neutron_e_eff(frac, sig):
    return np.sqrt(kf(frac)**2 + neutron_mass_eff(sig)**2)

def e_eff(baryon, frac, sig):
    return np.sqrt(kf(frac)**2 + neutron_mass_eff(sig)**2)

In [54]:
def proton_scalar_density(frac, sig):
    # returns scalar density n_s
    effective_mass = proton_mass_eff(sig)
    fermi_momentum = kf(frac) 
    eff_energy = proton_e_eff(frac, sig)
    
    coeff_1 = 1/(Pi**2)
    coeff_2 = rmf.g_sigma*effective_mass
    term_2 = eff_energy*fermi_momentum
    term_3 = np.log((eff_energy + fermi_momentum)/effective_mass)
    
    return coeff_1*coeff_2*(term_2 - effective_mass**2*term_3)

In [61]:
def scalar_density(baryon, frac, sig):
    effective_mass = mass_eff(baryon, sig)
    fermi_momentum = kf(frac)
    eff_energy = e_eff(baryon, frac, sig)
    
    coeff_1 = 1/(np.pi**2)
    coeff_2 = rmf.g_sigma*effective_mass
    term_2 = eff_energy * fermi_momentum 
    term_3 = np.log((eff_energy + fermi_momentum)/effective_mass)
    
    return coeff_1* coeff_2 * (term_2 - effective_mass**2*term_3)

In [62]:
def sigma_eom(proton_frac, neutron_frac, sig):
    tot = 0.0
    term_1 = sigma.mass**2*rmf.g_sigma + dU(sig)
    
    return term_1 - rmf.g_sigma*scalar_density(proton, proton_frac, sig) - rmf.g_sigma*scalar_density(neutron, neutron_frac, sig)

### Other things

In [65]:
def omega_eom(omega_field):
    return omega.mass**2 * omega_field - rmf.g_omega*nb

def rho_eom(proton_frac, neutron_frac, rho_field):
    return rho.mass**2 * rho_field - rmf.g_rho*proton.isospin*nb*proton_frac - rmf.g_rho*neutron.isospin*nb*neutron_frac

In [68]:
def baryon_chem_pot(baryon, frac, sig, omega_field, rho_field):
    return np.sqrt(kf(frac)**2 + mass_eff(baryon, sig)**2)\
        + rmf.g_omega*omega_field + baryon.isospin*rmf.g_rho*rho_field

def lepton_chem_pot(lepton, frac):
    return np.sqrt(kf(frac)**2 + lepton.mass**2)

In [73]:
def beta_equilibrium(proton_frac, electron_frac, neutron_frac, sig, omega_field, rho_field):
    proton_chem_pot = baryon_chem_pot(proton, proton_frac, sig, omega_field, rho_field)
    neutron_chem_pot = baryon_chem_pot(neutron, neutron_frac, sig, omega_field, rho_field)
    electron_chem_pot = lepton_chem_pot(electron, electron_frac)
    
    return neutron_chem_pot - proton_chem_pot - electron_chem_pot

In [70]:
def charge_cons(proton_frac, electron_frac):
    return proton_frac - electron_frac

def baryon_num_cons(proton_frac, neutron_frac):
    return 1 - proton_frac - neutron_frac 

## Goal for solving
1. We want to use Scipy nonlinear solvers
2. To pass this to Scipy:
    - Need numeric functions?

In [71]:
def fun(proton_frac, electron_frac, neutron_frac, sig, omega_field, rho_field):
    return [sigma_eom(proton_frac, neutron_frac, sig), omega_eom(omega_field),\
                rho_eom(proton_frac, neutron_frac, rho_field), beta_equilibrium(proton_frac, electron_frac, neutron_frac, sig, omega_field, rho_field),
                charge_cons(proton_frac, electron_frac), baryon_num_cons(proton_frac, neutron_frac)]

In [77]:
def fun_2(x):
    return [sigma_eom(x[0], x[2], x[3]), omega_eom(x[4]), rho_eom(x[1], x[3], x[5]),\
           beta_equilibrium(x[0], x[1], x[2], x[3], x[4], x[5]), charge_cons(x[0], x[1]),\
           baryon_num_cons(x[0], x[2])]

In [91]:
optimize.broyden1(fun_2, [5.0, 5.0, 5.99, 10.0, -2.0, 4.5])

<ipython-input-46-edcb88e89108>:2: RuntimeWarning: invalid value encountered in double_scalars
  return (3*np.pi**2*nb*frac)**(1/3)


NoConvergence: [ 5.    5.    5.99 10.   -2.    4.5 ]

## Re-writing system of equations 
1. Independent variables: $n_B, \sigma, \omega, \rho, k_p, k_n, k_e$
2. Meson equations of motion
$$
    m_\sigma^2 \sigma + bm_N g_\sigma^3 \sigma^2 + c g_\sigma^4 \sigma^3 = g_\sigma \sum_i n^s_i\\
    m_\omega^2 \omega = g_\omega n_B\\
    m_\rho^2 \rho = \sum_i I_{3i}g_\rho n_i
$$
3. Other conditions
$$
    n_B =  \frac{k_n^3}{3\pi^2} + \frac{k_p^3}{3\pi^2}\\
    k_p = k_e\\
    \sqrt{k_n^2 + {m_n^*}^2} = \sqrt{k_p^2 + {m_p^*}^2} + \sqrt{k_e^2 + m_e^2}
$$